Dataset Collection In this code, we obtain the dataset from three sources, the Enron dataset, Millersmile, and Nazario dataset.

In [ ]:
#Installation of allthe required libraries
!pip install requests
!pip install re
!pip install langdetect
!pip install nltk
!pip install autocorrect
!pip install googletrans==3.1.0a0 
!pip install wordninja

In [ ]:
#Importing all the necessary libraries 
import langdetect
import wordninja
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import nltk
import pandas as pd
import email
import time
import datetime

from autocorrect import Speller
from email import message_from_string
from nltk.corpus import words
from nltk.corpus import stopwords
from googletrans import Translator
from email import message_from_string
from dateutil import parser

In [ ]:
# Setting the stopwords in English language
stop_words = set(stopwords.words('english'))

Millersmile Data Extraction

In [ ]:
# Dataframe phish_emails_df created with columns - Name ,Subject and Text
phish_emails_df = pd.DataFrame(columns = ['Name', 'Subject', 'Text']) 

In [ ]:
#Web scraping
spam_email_data = []
count_spam_emails = 0
count_ham_emails = 51000
for i in range(296, 335 , 1):
    url = "http://www.millersmiles.co.uk/archives/"+str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    #Finding all occurrences with orange font and skipping the first one
    orange_fonts = soup.find_all('font',{'color':'orange'})[1:]
    for orange_font in orange_fonts:
        #Second orange_font is the name
        name = orange_font.get_text(strip=True)
        #Subject is a hyperlink
        subject = orange_font.find_next('a').get_text(strip=True)
        # Text is enclosed within blockquote
        text = orange_font.find_next('blockquote').get_text(strip=True)
        spam_email_data.append({'Name':name, 'Subject':subject, 'Text': text})
        count_spam_emails+=1
        if count_spam_emails >= count_ham_emails:
            break
    if count_spam_emails >= count_ham_emails:
            break
phish_emails_df = pd.DataFrame(spam_email_data)

In [ ]:
# Concatenation of subject and content into a new column text
phish_emails_df['text'] = phish_emails_df['Subject']+phish_emails_df['Text']
phish_emails_df['label'] = 1
phish_emails_df = phish_emails_df[['text', 'label']]

In [ ]:
# In dataset 2, we use a total of 2927 emails with training data length being 2342 and testing data length being around 585.
no_emails = 2927
train_data_len = 2342
phish_emails_df = phish_emails_df.sample(n = no_emails)
org_phish_train = phish_emails_df.sample(n = train_data_len)
org_phish_test = phish_emails_df.drop(org_phish_train.index)

Nazario Data Extraction

In [ ]:
# Load the Nazario dataset
emails_naz = pd.read_csv("Nazario.csv")
emails_naz['text'] = emails_naz['subject']+emails_naz['body']
emails_naz['label'] = 1
emails_naz_df = emails_naz[['text', 'label']]

Enron Data Extraction

In [ ]:
#emails.csv file is the Enron dataset stored in Kaggle
emails = pd.read_csv("emails.csv")

In [ ]:
def safe_parse_date(date_string):
    if not date_string:
        return None
    try:
        return parser.parse(date_string)
    except (ValueError, parser.ParserError):
        return None

df_email = pd.DataFrame([
    {
        'Content': complete_content.get_payload(),
        'Subject': d.get('Subject', 'No Subject'),
        'Date': d.get('Date'),  # Store the original date string
        'Label': 0
    }
    for i in range(emails.shape[0])
    for email in [emails.loc[i]['message']]
    for complete_content in [message_from_string(email)]
    for d in [dict(complete_content.items())]
])

# Convert 'Date' column to datetime
df_email['Date'] = pd.to_datetime(df_email['Date'].apply(safe_parse_date), errors='coerce')

# Extract year from the Date column
df_email['Year'] = df_email['Date'].dt.year

# Concatenation of subject and content into a new column text
df_email['text'] = df_email['Subject'] + df_email['Content']

In [ ]:
# Filter the DataFrame for emails from August 2001 onwards till December 2002
df_email['Date'] = pd.to_datetime(df_email['Date'])
start_date = pd.Timestamp('2001-08-01', tz='UTC')  # Or use the correct timezone
end_date = pd.Timestamp('2002-12-31 23:59:59', tz='UTC')

filtered_emails = df_email[(df_email['Date'] >= start_date) & (df_email['Date'] <= end_date)]
filtered_emails["label"]=0

In [ ]:
# Among a total of 22458 emails, 17967 emails are used for training and 4491 for testing 
no_of_emails = 22458
df_email = filtered_emails.sample(n=no_of_emails)
phish_emails_df_final = df_email[['text', 'label']]
num_of_train_emails = 17967
leg_train_df = phish_emails_df_final.head(num_of_train_emails)
leg_test_df = phish_emails_df_final.drop(leg_train_df.index)

In [ ]:
Dataset2_Train = pd.concat([org_phish_train, naz_spam_train, leg_train_df], axis = 0)
Dataset2_Test = pd.concat([org_phish_test, naz_spam_test, leg_test_df], axis = 0)

Tradional Preprocessing - The following fuctions are involved in the preprocessing stage
1) Removal of special characters
2) Conversion to lowercase
3) Removal of stop words
4) Removal of Numbers
5) Identification of URL, Numbers, Phone, Email
6) Detection and translation of languages (if not in English)
These preprocessing steps are followed to obtain the Dataset2_1.

In [ ]:
# Function to remove special characters from a given text string
def remove_special_characters(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'_+', '', cleaned_text)
    return cleaned_text  

In [ ]:
# Function to convert the text into lowercase
def convert_to_lowercase(text):
    words_list = [word.lower() for word in text.split()]
    cleaned_text = ' '.join(words_list)
    return cleaned_text

In [ ]:
#Function to remove stop words from text
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    words_list = [word.lower() for word in text.split() if word.lower() not in stop_words]
    cleaned_text = ' '.join(words_list)
    return cleaned_text

In [ ]:
# Function to remove numbers from the text
def remove_numbers(text):
    words = text.split()
    cleaned_text_list = []
    for word in words:
        word_list = [x for x in word if x.isdigit()!=True]
        cleaned_text_list.append("".join(word_list))
    cleaned_text = " ".join(cleaned_text_list)
    return cleaned_text

In [ ]:
# Function to identify attachment and urls in the text
def url_attachment_identification(text):
    url_pattern = r'\b(?:https?:\/\/)?(?:[\w-]+\.)+[a-z]{2,}(?:\/[^\s]*)?'
    attachment_pattern = r'\b\w+\.(pdf|docx|jpg|png|xls|xlsx|ppt|pptx|txt|zip)\b'
    email_pattern = r'\b[a-zA-Z0-9_.%+-]+@[a-zA-Z0-9.-]+\.[a-z|A-Z]{2,7}\b'
    phone_pattern = r'^\+?(\d{1,3})?[-.\s]?\(?\d{1,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,4}([-.\s]?(ext|x|#)\s?\d{1,5})?$' 

    cleaned_text = re.sub(email_pattern,"email",text)
    cleaned_text = re.sub(phone_pattern,"phone number",cleaned_text)
    cleaned_text = re.sub(url_pattern,"link",cleaned_text)
    cleaned_text = re.sub(attachment_pattern,"attachment",cleaned_text)
    return cleaned_text

In [ ]:
# Detection and translation of the email 
def detect_and_translate(text):
    try:
        detected_lang = langdetect.detect(text)
        if detected_lang == 'en':
            return text
        translator = Translator()
        translation = translator.translate(text, dest='en')
        return translation.text
    except Exception as e:
        return text

In [ ]:
# Function to preprocess the emails
def preprocess(text):
    if not isinstance(text, str):
        return ""
    cleaned_text = url_attachment_identification(text)
    cleaned_text = remove_special_characters(cleaned_text)
    cleaned_text = convert_to_lowercase(cleaned_text)
    cleaned_text = remove_stop_words(cleaned_text)
    cleaned_text = remove_numbers(cleaned_text)
    cleaned_text = detect_and_translate(cleaned_text)
    return cleaned_text

In [ ]:
# Apply preprocessing to the text column of dataframe df_email
Dataset2_Train['cleaned_text'] = Dataset2_Train["text"].apply(preprocess)
Dataset2_Test['cleaned_text'] = Dataset2_Test["text"].apply(preprocess)
Dataset2_1Train = Dataset2_Train#[["cleaned_text","label"]]
Dataset2_1Test = Dataset2_Test#[["cleaned_text","label"]]

Traditional + Split_Words + SpellChecker
Along with the traditional methods used, we also use the second level of preprocessing functions - 
1) Splitting of Words
2) Spelling Correction

In [ ]:
Dataset2_1Train.to_csv("Dataset2_1Train_new2.csv")
Dataset2_1Test.to_csv("Dataset2_1Test_new2.csv")
# Replace NaN values with an empty string
Dataset2_1Train = pd.read_csv("Dataset2_1Train_new2.csv")
Dataset2_1Train['cleaned_text'] = Dataset2_1Train['cleaned_text'].fillna('')
Dataset2_1Train.drop(columns=['Unnamed: 0'], inplace=True)
# Replace NaN values with an empty string
Dataset2_1Test = pd.read_csv("Dataset2_1Test_new2.csv")
Dataset2_1Test['cleaned_text'] = Dataset2_1Test['cleaned_text'].fillna('')
Dataset2_1Test.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
#Autocorrection of email
def autocorrect_email(text):
    check = Speller(lang="en")
    autocorrected_email = check(text)
    return autocorrected_email

In [ ]:
# Processing of Dataset1_1Train
for index, row in Dataset2_1Train.iterrows():
    email = row['cleaned_text']
    autocorrected_email = autocorrect_email(email)
    split_words = wordninja.split(autocorrected_email)
    cleaned_email = " ".join(split_words)
    Dataset2_1Train.loc[index, 'final_cleaned_text'] = cleaned_email

# Processing of Dataset1_1Test 
for index, row in Dataset2_1Test.iterrows():
    email = row['cleaned_text']
    autocorrected_email = autocorrect_email(email)
    split_words = wordninja.split(autocorrected_email)
    cleaned_email = " ".join(split_words)
    Dataset2_1Test.loc[index, 'final_cleaned_text'] = cleaned_email

# Extract necessary columns
Dataset2_2Train = Dataset2_1Train#[["final_cleaned_text","label"]]
Dataset2_2Test = Dataset2_1Test#[["final_cleaned_text","label"]]

In [ ]:
total_time = end_time - start_time
print(f"Total time to create Dataset 2_2 is {total_time} seconds")

In [ ]:
# Conversion of the dataframe into csv
#Dataset2_Train.to_csv("Dataset2_Train.csv")
#Dataset2_Test.to_csv("Dataset2_Test.csv")
#Dataset2_1Train.to_csv("Dataset2_1Train_new2.csv")
#Dataset2_1Test.to_csv("Dataset2_1Test_new2.csv")
#Dataset2_2Train.to_csv("Dataset2_2Train_new2.csv")
#Dataset2_2Test.to_csv("Dataset2_2Test_new2.csv")